In [1]:
import numpy as np 
import pandas as pd 
import scipy
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from skimage import transform
from __future__ import print_function, division

from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime
import sys
import os
from imageio import imread
from PIL import Image
import pickle

In [2]:
import rasterio #used to handle geoTiffs
from rasterio.plot import reshape_as_image
import matplotlib.pyplot as plt

In [3]:
def normalize(array):
    '''function that normalizes an array to 0-1 range (min max norm)'''
    minV, maxV = array.min(), array.max()
    return (array - minV) / (maxV - minV)

def rgb_geotiff_to_numpy(filepath):
    '''function that reads the RGB channels from
    a multispectral geoTiff and converst it to a RGB image'''
    with rasterio.open(filepath) as src:
        
        red = normalize(src.read(4))  
        green = normalize(src.read(3))  
        blue = normalize(src.read(2))
        
        rgb = np.stack([red, green, blue], axis=0)
        rgb = reshape_as_image(rgb)
        
    return rgb

In [4]:
# path_RGB = glob('D:/EcometricProject/Datasets/ROIs1970_fall_s2/*')
        
# file = np.random.choice(path_RGB)

# plt.imshow(rgb_geotiff_to_numpy(file))

In [5]:
def sar_geotiff_to_numpy(filepath):
    '''function that takes a SAR geoTiff and outputs a numpy array of the bands'''
    with rasterio.open(filepath) as src:
        
        vv = normalize(src.read(1))
        vh = normalize(src.read(2))
        
        stacked = np.stack([vv, vh], axis=0)
        stacked = reshape_as_image(stacked)
        
    return stacked

In [6]:
#GPU config
import tensorflow as tf
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

In [7]:
def load_data(batch_size=1, is_val = False):
       
        path_SAR = glob('D:/EcometricProject/Datasets/ROIs/ROIs1970_fall_s1/*')
        
        batch_images = np.random.choice(path_SAR, size=batch_size)

        imgs_A = []
        imgs_B = []
        for img_path in batch_images:
            img_B = sar_geotiff_to_numpy(img_path) #SAR image
            img_A = rgb_geotiff_to_numpy(img_path.replace('ROIs1970_fall_s1', 'ROIs1970_fall_s2')) #RGB image 


            # If training => do random flip , this is a trick to avoid overfitting 
            if not is_val and np.random.random() < 0.5:
                img_A = np.fliplr(img_A)
                img_B = np.fliplr(img_B)

            imgs_A.append(img_A)
            imgs_B.append(img_B)
            
        #imgs_A = np.array(imgs_A)/127.5 - 1.  #RGB images are already normalized
        imgs_A = np.array(imgs_A)
        imgs_B = np.array(imgs_B) #still researching normalization of SAR images
        
        return imgs_A, imgs_B

In [9]:
def load_batch(batch_size=1, is_val=False):
        
        path_SAR = glob('D:/EcometricProject/Datasets/ROIs/ROIs1970_fall_s1/*')
        
        n_batches=batch_size
        
        for i in range(n_batches-1):
            batch = path_SAR[i*batch_size:(i+1)*batch_size]
            imgs_A, imgs_B = [], []
            for img in batch:
                img_B = sar_geotiff_to_numpy(img) #SAR image
                img_A = rgb_geotiff_to_numpy(img.replace('ROIs1970_fall_s1', 'ROIs1970_fall_s2')) #RGB image
                
                # when training => do random flip , this is a trick to avoid overfitting 
                if not is_val and np.random.random() > 0.5:
                        img_A = np.fliplr(img_A)
                        img_B = np.fliplr(img_B)
                
                imgs_A.append(img_A)
                imgs_B.append(img_B)
                
            # normalizing the images 
            #imgs_A = np.array(imgs_A)/127.5 - 1. # RGB images are already normalized.
            imgs_A = np.array(imgs_A)
            imgs_B = np.array(imgs_B) #still researching normalization of SAR images

            yield imgs_A, imgs_B

def imread(path):
    #open image and conver it to float32.
    return np.array(Image.open(path).convert('RGB')).astype(np.float32)
                            

In [ ]:
def build_generator():
        """U-Net Generator"""

        def conv2d(layer_input, filters, f_size=4, bn=True):
            """Layers used during downsampling"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if bn:
                d = BatchNormalization(momentum=0.8)(d)
            return d

        def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
            """Layers used during upsampling"""
            u = UpSampling2D(size=2)(layer_input)
            u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
            if dropout_rate:
                u = Dropout(dropout_rate)(u)
            u = BatchNormalization(momentum=0.8)(u)
            u = Concatenate()([u, skip_input]) #skip connection
            return u
        
        d0 = Input(shape=(256,256,2))

        # Downsampling
        d1 = conv2d(d0, gf, bn=False)
        d2 = conv2d(d1, gf*2)
        d3 = conv2d(d2, gf*4)
        d4 = conv2d(d3, gf*8)
        d5 = conv2d(d4, gf*8)
        d6 = conv2d(d5, gf*8)
        d7 = conv2d(d6, gf*8)

        # Upsampling
        u1 = deconv2d(d7, d6, gf*8)
        u2 = deconv2d(u1, d5, gf*8)
        u3 = deconv2d(u2, d4, gf*8)
        u4 = deconv2d(u3, d3, gf*4)
        u5 = deconv2d(u4, d2, gf*2)
        u6 = deconv2d(u5, d1, gf)

        u7 = UpSampling2D(size=2)(u6)
        output_img = Conv2D(channels, kernel_size=4, strides=1, padding='same', activation='tanh')(u7)

        return Model(d0, output_img)

In [ ]:
def build_discriminator():
        # a small function to make one layer of the discriminator
        def d_layer(layer_input, filters, f_size=4, bn=True):
            """Discriminator layer"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if bn:
                d = BatchNormalization(momentum=0.8)(d)
            return d

        img_A = Input(shape=img_shape)
        img_B = Input(shape=(256,256,2))

        # Concatenate image and conditioning image by channels to produce input
        combined_imgs = Concatenate(axis=-1)([img_A, img_B])

        d1 = d_layer(combined_imgs, df, bn=False)
        d2 = d_layer(d1, df*2)
        d3 = d_layer(d2, df*4)
        d4 = d_layer(d3, df*8)

        validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d2)

        return Model([img_A, img_B], validity)

In [ ]:
# Input shape
img_rows = 256
img_cols = 256
channels = 3
img_shape = (img_rows, img_cols, channels)


# Calculate output shape of D (PatchGAN)
patch = int(img_rows / 2**2)
disc_patch = (patch, patch, 1)

# Number of filters in the first layer of G and D
gf = 64
df = 64

optimizer = Adam(0.0002, 0.5)

# Build and compile the discriminator
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

# Build the generator
generator = build_generator()

# Input images and their conditioning images
img_A = Input(shape=img_shape)
img_B = Input(shape=(256,256,2))

# By conditioning on B generate a fake version of A
fake_A = generator(img_B)

# For the combined model we will only train the generator
discriminator.trainable = False

# Discriminators determines validity of translated images / condition pairs
valid = discriminator([fake_A, img_B])

combined = Model(inputs=[img_A, img_B], outputs=[valid, fake_A])
combined.compile(loss=['mse', 'mae'],
                              loss_weights=[1, 100],
                              optimizer=optimizer)

In [ ]:
def show_images( dataset_name,epoch, batch_i):
        '''function to take 3 images and show the results'''
        r, c = 2, 3

        imgs_A, imgs_B = load_data(batch_size=3)
        fake_A = generator.predict(imgs_B)

        gen_imgs = np.concatenate([fake_A, imgs_A])

        titles = ['Output', 'Ground Truth']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt])
                axs[i, j].set_title(titles[i])
                axs[i,j].axis('off')
                cnt += 1
        plt.show()
        plt.close()

In [ ]:
def train( dataset_name,epochs, batch_size=1, show_interval=10):

        start_time = datetime.datetime.now()

        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + disc_patch)
        fake = np.zeros((batch_size,) + disc_patch)

        for epoch in range(epochs):
            for batch_i, (imgs_A, imgs_B) in enumerate(load_batch(batch_size)):

                #  Train Discriminator

                # Condition on B and generate a translated version
                fake_A = generator.predict(imgs_B)
                
                # Train the discriminators (original images = real / generated = Fake)
                d_loss_real = discriminator.train_on_batch([imgs_A, imgs_B], valid)
                d_loss_fake = discriminator.train_on_batch([fake_A, imgs_B], fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
                
                

               
                #  Train Generator
                g_loss = combined.train_on_batch([imgs_A, imgs_B], [valid, imgs_A])

                elapsed_time = datetime.datetime.now() - start_time
                
            # Plot the progress
            if epoch%10==0:
                  print ("[Epoch %d/%d]  [D loss: %f, acc: %3d%%] [G loss: %f] time: %s" % (epoch, epochs,    
                                                                        d_loss[0], 100*d_loss[1],
                                                                        g_loss[0],
                                                                        elapsed_time))
            # If at show interval => show generated image samples
            if epoch % show_interval == 0:
                    show_images(dataset_name,epoch, batch_i)

In [ ]:
# train the model
with tf.device(tf.DeviceSpec(device_type="GPU", device_index=0)):
    train("cityscapes",epochs=1000, batch_size=4, show_interval=10)

In [ ]:
with open('temp.pkl', 'wb') as file:
        pickle.dump(generator, file)

In [ ]:
with open('temp.pkl', 'rb') as file:
        generator1 = pickle.load(file)

In [ ]:
def show_images_1( dataset_name,epoch, batch_i, generator1):
        
        r, c = 3, 3

        imgs_A, imgs_B = load_data(batch_size=3)
        fake_A = generator1.predict(imgs_B)

        gen_imgs = np.concatenate([imgs_B, fake_A, imgs_A])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Input', 'Output', 'Ground Truth']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt])
                axs[i, j].set_title(titles[i])
                axs[i,j].axis('off')
                cnt += 1
        plt.show()
        plt.close()

In [ ]:
show_images_1(1,1,1,generator1)